## Load Required Packages

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import string
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
import pandas as pd

### Recommend Recipes based on User Input

### Parse HTML to Extract Ingredient List

In [50]:
def get_ingredients(url):
    # Make a GET request to fetch the raw HTML content
    html_content = requests.get(url).text
    # Parse the html content
    soup = BeautifulSoup(html_content, "lxml")
    if any(x in url for x in ('delish', 'countryliving', 'goodhousekeeping')):
        ingredients = [ li.text.replace("\t", "").replace("\n", "").strip() for li in soup.find_all("div", attrs={"class": "ingredient-item"})]
    elif any(x in url for x in ('cookinglight','myrecipes')):
        ingredients = str(soup.find("div", attrs={"class": "ingredients"})).replace("<li>", "").replace("</li>", "").split('\n')[2:-2]
    elif 'damn' in url:
        ingredients = [ li.text for li in soup.find_all("li", attrs={"itemprop": "ingredients"})]
    else:
        ingredients = []
    return ingredients

In [51]:
ingredients1 = get_ingredients('http://www.myrecipes.com/recipe/turkey-bean-chili')
ingredients2 = get_ingredients('https://www.countryliving.com/food-drinks/a23336101/seared-sausage-with-cabbage-and-pink-lady-apples-recipe/')

In [31]:
def process_ingredients(ingredients):
    ingredients = [x.lower() for x in ingredients]
    ingredients = [re.sub(r"([2-9])",r"\1 ", text) for text in ingredients]
    return ingredients

In [32]:
ingredients1 = process_ingredients(ingredients1)

In [33]:
ingredients2 = process_ingredients(ingredients2)

In [34]:
combined_ingredients = ingredients1+ingredients2

## tag each ingredient by type

In [47]:
tagged_ingredients = pd.read_csv(r"Documents/cooking project/tagged_ingredients.csv")

In [48]:
def grocery_tagger(ingredient_list, tags):
    # Define the sorter
    sorter = ['vegetable','fruit','meat','grain','condiment','canned good',
             'spice','baking','frozen','dairy']

    ingredient_list_df = pd.DataFrame(ingredient_list)
    ingredient_list_df['category'] = 0
    ingredient_list_df['name'] = 0
    for index, row in ingredient_list_df.iterrows():
        category = 'unknown'
        name = 'unknown'
        for index2, row2 in tags.iterrows():
            if row2['Name'] in row[0]:
                category = row2['Category']
                name = row2['Name']
                break
        ingredient_list_df.loc[index, 'category'] = category
        ingredient_list_df.loc[index, 'name'] = name
    ingredient_list_df = ingredient_list_df.sort_values('name')   
    true_sort = [s for s in sorter if s in ingredient_list_df.category.unique()]
    ingredient_list_df = ingredient_list_df.set_index('category').loc[true_sort].reset_index()
    return ingredient_list_df

In [49]:
grocery_tagger(combined_ingredients, tagged_ingredients)

,category,0,name
0,vegetable,"1/2 head red cabbage, cut into 1/2 -inch-thick...",cabbage
1,vegetable,1/2 cup chopped fresh cilantro,cilantro
2,vegetable,"1/2 red onion, sliced",onion
3,vegetable,1 cup prechopped red onion,onion
4,vegetable,1/3 cup chopped seeded poblano pepper (about 1),pepper
5,fruit,"2 pink lady apples, halved",apple
6,fruit,6 lime wedges,lime
7,meat,6 small sweet italian sausage links (about 1 ...,sausage
8,meat,1 1/4 pounds ground turkey,turkey
9,condiment,1 c.fresh apple cider,cider
